# **사전 준비**

In [ ]:
!pip install langchain langchain_openai langchain_community pypdf faiss-cpu

In [ ]:
from google.colab import drive
import os

# 먼저 구글 드라이브 마운트
drive.mount('/content/drive')

In [ ]:

import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv("/content/.env")

# 환경 변수에서 API 키 가져오기
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

file_path = (
    "파일 경로"
)
loader = PyPDFLoader(file_path)

doc_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
docs = loader.load_and_split(doc_splitter)

# **리트리버 구축**

In [ ]:
# OpenAI의 임베딩 모델 사용
from langchain.embeddings import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
# FAISS 라이브러리 임포트
from langchain_community.vectorstores import FAISS

# FAISS DB 생성 후 저장
faiss_store = FAISS.from_documents(docs, embeddings)
faiss_store.save_local("/content/DB")

In [ ]:
# 저장된 DB 경로 지정 후 DB 로드
persist_directory = "/content/DB"

# allow_dangerous_deserialization=True
# FAISS는 내부적으로 Pickle 파일(index.pkl) 을 로드
# pickle은 임의의 Python 객체를 복원하는 방식이기 때문에 보안 위험이 있다
# LangChain에서는 기본적으로 이 옵션이 꺼져 있다
vectordb = FAISS.load_local(persist_directory, embeddings=embedding, allow_dangerous_deserialization=True)

In [ ]:
# FAISS 리트리버 생성
faiss_retriever = vectordb.as_retriever(search_kwargs={"k": 2})

# **질의**

In [ ]:
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain_openai import ChatOpenAI

# 관련 있는 문서 수집 후 챗 GPT로 최종 답변까지 수행
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=0.2, model="gpt-4o")
    chain_type="stuff"
    retriever=faiss_retriever,
    return_source_documents=True,
)

In [ ]:
qa_chain.invoke("이 회사가 발행한 주식의 총 발행량이 어느 정도야?")